# This didn't work out that well (See topics)

In [17]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, cross_val_predict
from scipy.sparse import hstack
from scipy.special import logit, expit

from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF
from sklearn.metrics.pairwise import cosine_similarity

import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import re
import nltk
import string

import gensim

import pickle

# Initiate Class Names and Open Files

In [31]:
class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

train = pd.read_csv('/Users/randy/Documents/GitHub/Twitch_Chat_Harassment/toxic_comment_classification/jigsaw-toxic-comment-classification-challenge/cleaned_train.csv').fillna(' ')
test = pickle.load(open('/Users/randy/Documents/GitHub/Twitch_Chat_Harassment/chat_logs/callofduty_chat.p','rb'))

# Train Feature Engineering

In [32]:
#remove pucnt
def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    return text

train['comment_text'] = train['comment_text'].apply(lambda x: remove_punct(x))
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,explanation why the edits made under my userna...,0,0,0,0,0,0
1,000103f0d9cfb60f,d aww he matches this background colour i am ...,0,0,0,0,0,0
2,000113f07ec002fd,hey man i am really not trying to edit war it ...,0,0,0,0,0,0
3,0001b41b1c6bb37e,more i cannot make any real suggestions on imp...,0,0,0,0,0,0
4,0001d958c54c6e35,you sir are my hero any chance you remember wh...,0,0,0,0,0,0


In [33]:
#remove non-ascii characters
def remove_non_ascii(text):
    text  = "".join([char for char in text if char in string.printable])
    text = re.sub('[0-9]+', '', text)
    return text

train['comment_text'] = train['comment_text'].apply(lambda x: remove_non_ascii(x))
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,explanation why the edits made under my userna...,0,0,0,0,0,0
1,000103f0d9cfb60f,d aww he matches this background colour i am ...,0,0,0,0,0,0
2,000113f07ec002fd,hey man i am really not trying to edit war it ...,0,0,0,0,0,0
3,0001b41b1c6bb37e,more i cannot make any real suggestions on imp...,0,0,0,0,0,0
4,0001d958c54c6e35,you sir are my hero any chance you remember wh...,0,0,0,0,0,0


# Test Engineering

In [34]:
ar = np.array(range(0,len(test)))

In [35]:
test['id'] = ar

In [36]:
list_sentences_train = train['comment_text']
list_sentences_test = test['message']

In [37]:
all_text = pd.concat([list_sentences_train, list_sentences_test])

# Apply TF-IDF Vectorizer

In [44]:
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    ngram_range=(1, 1),
    max_features=50000)
word_vectorizer.fit(all_text)
train_word_features = word_vectorizer.transform(train)
test_word_features = word_vectorizer.transform(test_text)

In [89]:
# Acronynms: Latent Semantic Analysis (LSA) is just another name for 
#  Signular Value Decomposition (SVD) applied to Natural Language Processing (NLP)
n = 20

lsa_train = TruncatedSVD(n)
train_doc_topic = lsa_train.fit_transform(train_doc_word)
lsa_train.explained_variance_ratio_

array([0.04311094, 0.03398691, 0.0338451 , 0.03338573, 0.02805996,
       0.02549024, 0.02536562, 0.0244332 , 0.0216698 , 0.02165827,
       0.01878569, 0.01869414, 0.01795428, 0.01691661, 0.01630048,
       0.01562888, 0.01504708, 0.01208367, 0.01331639, 0.01234063])

In [90]:
topic_word = pd.DataFrame(lsa_train.components_.round(3),
             index = [f'component_{i}' for i in range (n)],
             columns = vectorizer.get_feature_names())
topic_word

,__,___,____,_____,______,_______,________,_________,___________,_____________,...,조선인민군,척뉴넘,천리마군,칠지도,편집,ﬂute,ｳｨｷﾍﾟﾃﾞｨｱ,𐌰𐌹,𐌰𐌿,𐌴𐌹
component_0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
component_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
component_2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
component_3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
component_4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
component_5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
component_6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
component_7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
component_8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
component_9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [91]:
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [92]:
display_topics(lsa_train, vectorizer.get_feature_names(), 5)


Topic  0
fuck, yourselfgo, ass, dog, bitches

Topic  1
lol, lunchables, donkey, imma, nice

Topic  2
pig, fat, jew, bullshit, wikipedia

Topic  3
fat, jew, cunt, fucking, jewfat

Topic  4
hi, moron, wikipedia, hate, page

Topic  5
nigger, die, shit, fag, cunt

Topic  6
die, fag, shit, di, edie

Topic  7
shit, ass, faggot, eat, user

Topic  8
fish, freedom, bullshit, fucksex, wiki

Topic  9
bark, fish, freedom, fucksex, twat

Topic  10
suck, mexicans, dick, cock, ass

Topic  11
oh, noes, lawdy, bob, saget

Topic  12
hate, wikipedia, page, article, like

Topic  13
ass, faggot, huge, block, gay

Topic  14
faggot, huge, block, suck, fuck

Topic  15
block, wikipedia, page, talk, article

Topic  16
balls, fucksex, bullshit, freedom, wiki

Topic  17
wikipedia, article, page, like, talk

Topic  18
gay, super, bunksteve, moron, hate

Topic  19
tacos, aids, sex, teabag, sexsex


In [85]:
Vt = pd.DataFrame(train_doc_topic.round(5),
             index = train,
             columns = [f'component_{i}' for i in range (n)])

In [93]:
Vt[Vt['component_0'] > 2].head(3)

,component_0,component_1,component_2,component_3,component_4,component_5,component_6,component_7,component_8,component_9,...,component_30,component_31,component_32,component_33,component_34,component_35,component_36,component_37,component_38,component_39
"(0021fe88bc4da3e6, my band page deletion you thought i was gone deleting the comment i posted on your talk page does not delete my feelings for your hasty decision to delete my page you sir are still a cock sucking douche fuck sit back for a second and think about what your life has become done that did not take long did it nope because as i am most certainly aware your life is a shitstorm of sitting in front of the computer masturbating to fictional creatures your attempts to get rid of me are mediocre at best you are constantly sucking the dick of failure you do not want a personal attack huh well too bad fuckcock mcdickerson you have got one from me you can go ahead and delete my profile from wikipedia i will just make another one and come right back to yrthiss talk page and insult the dick off of you how could you shatter the dreams of an innocent eighteen year old college freshman trying to make a name for his band does that make you happy fucking with people because you are an overweight single old man in a dead end job did you spot that perhaps someone else was going to follow his dreams and you were trying to hold him back so somebody else could suffer like you yes you did i do not make empty threats so i wo not be saying anything along the lines of i will hurt you or i will eat the children from within your sister womb but i will say that you are a asshole son of a bitch mother fucking cock sucker so go eat some more food and drown your sorrows you premature ejaculating bald headed fuck you should do something nice for yourself maybe go grab a couple of horny goat weeds from your local convenience store and jack off for a little longer than three minutes tonight sincerely an asshole that better than you in every way, 1, 0, 1, 0, 0, 0)",2.10350,0.01456,0.00123,0.01212,0.53504,0.01370,0.07784,0.11030,0.00556,0.00033,...,-3.11053,-0.25006,-0.07345,-0.13830,-0.03547,0.06294,0.60976,-0.05436,0.01985,-0.28669
"(00c59720428c985a, fuck you fuck you award go fuck yourself, 1, 0, 1, 0, 1, 0)",2.94745,-0.00069,-0.00035,-0.01939,-0.05513,-0.05667,-0.01430,-0.03885,-0.00011,-0.00000,...,-0.02174,-0.00217,0.00017,-0.00009,0.00067,-0.04611,0.00473,0.00080,0.01318,-0.00488
"(02c6e41e4b317ac3, would not be the first time bitch fuck you i will find out where you live sodomize your wife and then burn your house down fuck you you fucking queer, 1, 1, 1, 1, 1, 1)",2.02196,-0.00008,-0.00016,-0.00579,-0.03162,-0.02765,0.01181,-0.00149,0.00009,-0.00001,...,-0.09990,0.00188,0.00047,0.00575,0.02541,0.04789,-0.00208,0.03252,0.04645,0.03200


In [94]:
Vt[Vt['component_2'] > 2].head(3)

component_0  component_1  \
(13738e570aeab6f8, regrettably i have to agree ...      0.11793      0.01823   
(1a6002dec4fb8851, up to your old tricks i see ...      1.10294      0.01196   
(38a501b9457b0dc3, disgusting hideously ugly fa...      0.00785      0.00200   

                                                    component_2  component_3  \
(13738e570aeab6f8, regrettably i have to agree ...      3.00359      0.01468   
(1a6002dec4fb8851, up to your old tricks i see ...      2.00157      0.01346   
(38a501b9457b0dc3, disgusting hideously ugly fa...      2.00052      0.71599   

                                                    component_4  component_5  \
(13738e570aeab6f8, regrettably i have to agree ...      0.42617      0.03855   
(1a6002dec4fb8851, up to your old tricks i see ...      0.75097      0.39577   
(38a501b9457b0dc3, disgusting hideously ugly fa...      0.02084      0.00034   

                                                    component_6  component_7  \
(13738e570aeab6f8, regrettably i have to agree ...      0.04900      0.11150   
(1a6002dec4fb8851, up to your old tricks i see ...      0.52012      1.97512   
(38a501b9457b0dc3, disgusting hideously ugly fa...      0.00375      0.00112   

                                                    component_8  component_9  \
(13738e570aeab6f8, regrettably i have to agree ...      0.00964      0.00066   
(1a6002dec4fb8851, up to your old tricks i see ...      0.00501      0.00039   
(38a501b9457b0dc3, disgusting hideously ugly fa...      0.00034      0.00002   

                                                    ...  component_30  \
(13738e570aeab6f8, regrettably i have to agree ...  ...      -1.78023   
(1a6002dec4fb8851, up to your old tricks i see ...  ...      -0.30647   
(38a501b9457b0dc3, disgusting hideously ugly fa...  ...      -0.23006   

                                                    component_31  \
(13738e570aeab6f8, regrettably i have to agree ...      -0.21665   
(1a6002dec4fb8851, up to your old tricks i see ...      -0.05481   
(38a501b9457b0dc3, disgusting hideously ugly fa...      -0.01875   

                                                    component_32  \
(13738e570aeab6f8, regrettably i have to agree ...      -0.10155   
(1a6002dec4fb8851, up to your old tricks i see ...      -0.02097   
(38a501b9457b0dc3, disgusting hideously ugly fa...      -0.00815   

                                                    component_33  \
(13738e570aeab6f8, regrettably i have to agree ...       0.91837   
(1a6002dec4fb8851, up to your old tricks i see ...       0.45933   
(38a501b9457b0dc3, disgusting hideously ugly fa...      -0.01369   

                                                    component_34  \
(13738e570aeab6f8, regrettably i have to agree ...       0.10967   
(1a6002dec4fb8851, up to your old tricks i see ...       0.08655   
(38a501b9457b0dc3, disgusting hideously ugly fa...      -0.00216   

                                                    component_35  \
(13738e570aeab6f8, regrettably i have to agree ...       0.09691   
(1a6002dec4fb8851, up to your old tricks i see ...       0.19083   
(38a501b9457b0dc3, disgusting hideously ugly fa...       0.03698   

                                                    component_36  \
(13738e570aeab6f8, regrettably i have to agree ...       0.13858   
(1a6002dec4fb8851, up to your old tricks i see ...       0.15076   
(38a501b9457b0dc3, disgusting hideously ugly fa...       0.00093   

                                                    component_37  \
(13738e570aeab6f8, regrettably i have to agree ...      -0.35977   
(1a6002dec4fb8851, up to your old tricks i see ...      -0.04172   
(38a501b9457b0dc3, disgusting hideously ugly fa...       0.00438   

                                                    component_38  component_39  
(13738e570aeab6f8, regrettably i have to agree ...       1.04884       2.29761  
(1a6002dec4fb8851, up to your old tricks i see ...       0.24262       1.07133  


In [95]:
Vt[Vt['component_3'] > 0.8].head(3)

,component_0,component_1,component_2,component_3,component_4,component_5,component_6,component_7,component_8,component_9,...,component_30,component_31,component_32,component_33,component_34,component_35,component_36,component_37,component_38,component_39
"(02015f8b6caeac9f, no not really we may ask that the mention of fat being the fire source of the cremation of millions be reconsidered though along with a few other items the fat cremation wiki fact is citable www hdot emory u no less lipstadt but doubtful if the same science was applied to the holocaust as say the tinfoilers or flat earthers the deniers would be overjoyed be careful as to who gets the nutty fringe tinfoil label in the end you get the permits and we will bring the shovels, 0, 0, 0, 0, 0, 0)",0.01281,0.00139,0.00100,1.43037,0.02059,-0.00208,0.00229,0.00224,0.00038,0.00003,...,-0.17577,-0.01411,-0.00812,-0.00890,-0.00015,0.04170,0.00882,-0.00276,-0.00575,-0.00299
"(040a95741d7c8d91, non involved people most definitely cannot understand yeshivish a nice example of yeshivish is the english yated neeman which can be found online at many of their articles simply cannot be understood by a non jew or a non orthodox jew that is a simple fact it is english but those few hebrew and aramaic and yiddish words mixed in make many articles totally unreadable for the average person i removed the disputed tag it has been there for months and barely anything was done about the article since so i presumed it was safe to remove it talk, 0, 0, 0, 0, 0, 0)",0.01581,0.00470,0.00118,1.40019,0.09692,0.00015,0.00957,0.01752,0.00162,0.00011,...,-1.22386,-0.09873,-0.04569,-0.08994,-0.01128,-0.06042,-0.06861,0.02671,-0.07798,-0.09564
"(0e37a1d465496716, henrietta van laer says no it not that having an agenda at all is a problem for wikipedia wikipedia has a very clear and unconcealable agenda pro socialists liberals greens and gays determined to vilify and demonize conservatives it is a completely biassed source its articles on so called left wing parties are blatant whitewash while those on so called right wing parties are nothing but vilification most of it based on flimsy third hand sources and malicious allegations with no basis in fact instead of detailing their policies wikipedia provides a compendium of prejudice the frequent denials that you have a point of view or that you are biassed are not at all convincing particularly since the whole slant is maintained by a policy of rigorous censorship pov in wiki speak different pov to us the wiki police i also get the impression that wikipeople are pretty young there is no sign of maturity or experience of the real world in their political views under conservative we find the definition that these parties are merely in favour of the rich how naive can you get i do not know what the term fat cat means in the usa but here in britain it has a very specific meaning i e the people employed in the public sector who get salaries of times the average naturally they are inclined to vote labour and can also legally donate large sums to that party then there are a million more people employed in useless quangos which would be abolished by a right wing tax cutting party many of them earn two or three times the average wage and like the fat cats they get generous pension packages too added to the votes of the millions of people who live long term on state benefits this amounts to a huge vested interest in the socialist programme on the part of people who are neither honest joe workers or low paid but wikipedia is still living in a world of student politics with a poster of che guevara on the wall his hair streaming in the wind, 0, 0, 0, 0, 0, 0)",0.19066,0.00951,0.00202,1.43289,1.28155,0.01501,0.04080,0.02308,0.00508,0.00033,...,0.37715,0.03397,0.00200,0.16322,0.05176,0.10593,-0.04984,-0.02767,0.13896,0.32580


In [96]:
Vt[Vt['component_4'] > 0.8].head(3)

,component_0,component_1,component_2,component_3,component_4,component_5,component_6,component_7,component_8,component_9,...,component_30,component_31,component_32,component_33,component_34,component_35,component_36,component_37,component_38,component_39
"(0005300084f90edc, fair use rationale for thanks for uploading i notice the image page specifies that the image is being used under fair use but there is no explanation or rationale as to why its use in wikipedia articles constitutes fair use in addition to the boilerplate fair use template you must also write out on the image description page a specific explanation or rationale for why using this image in each article is consistent with fair use please go to the image description page and edit it to include a fair use rationale if you have uploaded other fair use media consider checking that you have specified the fair use rationale on those pages too you can find a list of image pages you have edited by clicking on the my contributions link it is located at the very top of any wikipedia page when you are logged in and then selecting image from the dropdown box note that any fair use images uploaded after may and lacking such an explanation will be deleted one week after they have been uploaded as described on criteria for speedy deletion if you have any questions please ask them at the media copyright questions page thank you talk contribs unspecified source for thanks for uploading i noticed that the file description page currently does not specify who created the content so the copyright status is unclear if you did not create this file yourself then you will need to specify the owner of the copyright if you obtained it from a website then a link to the website from which it was taken together with a restatement of that website terms of use of its content is usually sufficient information however if the copyright holder is different from the website publisher then their copyright should also be acknowledged as well as adding the source please add a proper copyright licensing tag if the file does not have one already if you created took the picture audio or video then the tag can be used to release it under the gfdl if you believe the media meets the criteria at wikipedia fair use use a tag such as or one of the other tags listed at wikipedia image copyright tagsfair use see wikipedia image copyright tags for the full list of copyright tags that you can use if you have uploaded other files consider checking that you have specified their source and tagged them too you can find a list of files you have uploaded by following this link unsourced and untagged images may be deleted one week after they have been tagged as described on criteria for speedy deletion if the image is copyrighted under a non free license per wikipedia fair use then the image will be deleted hours after if you have any questions please ask them at the media copyright questions page thank you talk contribs , 0, 0, 0, 0, 0, 0)",0.27045,0.01796,0.00242,0.00818,2.58699,0.03351,0.07544,0.04554,0.01312,0.00001,...,-7.92606,-0.59882,-0.28207,-0.71619,-0.15566,-0.43978,-0.79998,0.27346,-1.71955,-1.64259
"(0016e01b742b8da3, notability of rurika kasuga a tag has been placed on rurika kasuga requesting that it be speedily deleted from wikipedia this has been done because the article seems to be about a person group of people band club company or web content but it does not indicate how or why the subject is notable that is why an article about that subject should be included in wikipedia under the criteria for speedy deletion articles that do not assert notability may be deleted at any time please see the guidelines for what is generally accepted as notable and if you can indicate why the subject of this article is notable you may contest the tagging to do this add on the top of the page below the existing db tag and leave a note on the article talk page explaining your position please do not remove the speedy del

In [97]:
Vt[Vt['component_5'] > 2].head(3)

component_0  component_1  \
(0b4797b2dff0afaa, no seriousy the only thing i...      2.21397      0.03143   
(0c7c341727488579, do go fuck off bastard do yy...     51.96472      0.04543   
(130bf0cafb548a89, supertr ll will live forever...      0.09289      0.00360   

                                                    component_2  component_3  \
(0b4797b2dff0afaa, no seriousy the only thing i...      0.00322      0.02274   
(0c7c341727488579, do go fuck off bastard do yy...     -0.00235     -0.23450   
(130bf0cafb548a89, supertr ll will live forever...      0.00108      0.00381   

                                                    component_4  component_5  \
(0b4797b2dff0afaa, no seriousy the only thing i...      1.43870      5.21969   
(0c7c341727488579, do go fuck off bastard do yy...     -1.16349     23.03323   
(130bf0cafb548a89, supertr ll will live forever...      0.38263      5.79890   

                                                    component_6  component_7  \
(0b4797b2dff0afaa, no seriousy the only thing i...      0.13810     -0.92067   
(0c7c341727488579, do go fuck off bastard do yy...     30.74150     42.18014   
(130bf0cafb548a89, supertr ll will live forever...     19.35348     -6.00468   

                                                    component_8  component_9  \
(0b4797b2dff0afaa, no seriousy the only thing i...      0.01299      0.00075   
(0c7c341727488579, do go fuck off bastard do yy...      0.02530     -0.00056   
(130bf0cafb548a89, supertr ll will live forever...      0.00036     -0.00003   

                                                    ...  component_30  \
(0b4797b2dff0afaa, no seriousy the only thing i...  ...      -4.33114   
(0c7c341727488579, do go fuck off bastard do yy...  ...       4.35468   
(130bf0cafb548a89, supertr ll will live forever...  ...      -0.16233   

                                                    component_31  \
(0b4797b2dff0afaa, no seriousy the only thing i...      -0.38929   
(0c7c341727488579, do go fuck off bastard do yy...       0.15484   
(130bf0cafb548a89, supertr ll will live forever...       0.13831   

                                                    component_32  \
(0b4797b2dff0afaa, no seriousy the only thing i...      -0.13212   
(0c7c341727488579, do go fuck off bastard do yy...       0.15289   
(130bf0cafb548a89, supertr ll will live forever...      -0.15915   

                                                    component_33  \
(0b4797b2dff0afaa, no seriousy the only thing i...       0.36043   
(0c7c341727488579, do go fuck off bastard do yy...      -0.13926   
(130bf0cafb548a89, supertr ll will live forever...       0.69762   

                                                    component_34  \
(0b4797b2dff0afaa, no seriousy the only thing i...       0.02854   
(0c7c341727488579, do go fuck off bastard do yy...      -0.14374   
(130bf0cafb548a89, supertr ll will live forever...       0.32527   

                                                    component_35  \
(0b4797b2dff0afaa, no seriousy the only thing i...      -0.49358   
(0c7c341727488579, do go fuck off bastard do yy...      -0.68770   
(130bf0cafb548a89, supertr ll will live forever...      -0.19369   

                                                    component_36  \
(0b4797b2dff0afaa, no seriousy the only thing i...       0.19122   
(0c7c341727488579, do go fuck off bastard do yy...       1.12707   
(130bf0cafb548a89, supertr ll will live forever...      -0.57568   

                                                    component_37  \
(0b4797b2dff0afaa, no seriousy the only thing i...      -0.47588   
(0c7c341727488579, do go fuck off bastard do yy...      -0.48718   
(130bf0cafb548a89, supertr ll will live forever...       0.16828   

                                                    component_38  component_39  
(0b4797b2dff0afaa, no seriousy the only thing i...       0.98000       0.96445  
(0c7c341727488579, do go fuck off bastard do yy...      -0.91849       0.34819  


In [98]:
Vt[Vt['component_6'] > 2].head(3)

component_0  component_1  \
(04bbc7a4ebe7919a, about einstein my search of ...      0.03980      0.00867   
(0c7c341727488579, do go fuck off bastard do yy...     51.96472      0.04543   
(130bf0cafb548a89, supertr ll will live forever...      0.09289      0.00360   

                                                    component_2  component_3  \
(04bbc7a4ebe7919a, about einstein my search of ...      0.00113      0.00755   
(0c7c341727488579, do go fuck off bastard do yy...     -0.00235     -0.23450   
(130bf0cafb548a89, supertr ll will live forever...      0.00108      0.00381   

                                                    component_4  component_5  \
(04bbc7a4ebe7919a, about einstein my search of ...      0.32399      0.98884   
(0c7c341727488579, do go fuck off bastard do yy...     -1.16349     23.03323   
(130bf0cafb548a89, supertr ll will live forever...      0.38263      5.79890   

                                                    component_6  component_7  \
(04bbc7a4ebe7919a, about einstein my search of ...      3.27065     -0.96929   
(0c7c341727488579, do go fuck off bastard do yy...     30.74150     42.18014   
(130bf0cafb548a89, supertr ll will live forever...     19.35348     -6.00468   

                                                    component_8  component_9  \
(04bbc7a4ebe7919a, about einstein my search of ...      0.00463      0.00024   
(0c7c341727488579, do go fuck off bastard do yy...      0.02530     -0.00056   
(130bf0cafb548a89, supertr ll will live forever...      0.00036     -0.00003   

                                                    ...  component_30  \
(04bbc7a4ebe7919a, about einstein my search of ...  ...      -3.42979   
(0c7c341727488579, do go fuck off bastard do yy...  ...       4.35468   
(130bf0cafb548a89, supertr ll will live forever...  ...      -0.16233   

                                                    component_31  \
(04bbc7a4ebe7919a, about einstein my search of ...      -0.29025   
(0c7c341727488579, do go fuck off bastard do yy...       0.15484   
(130bf0cafb548a89, supertr ll will live forever...       0.13831   

                                                    component_32  \
(04bbc7a4ebe7919a, about einstein my search of ...      -0.14380   
(0c7c341727488579, do go fuck off bastard do yy...       0.15289   
(130bf0cafb548a89, supertr ll will live forever...      -0.15915   

                                                    component_33  \
(04bbc7a4ebe7919a, about einstein my search of ...      -0.11824   
(0c7c341727488579, do go fuck off bastard do yy...      -0.13926   
(130bf0cafb548a89, supertr ll will live forever...       0.69762   

                                                    component_34  \
(04bbc7a4ebe7919a, about einstein my search of ...      -0.10020   
(0c7c341727488579, do go fuck off bastard do yy...      -0.14374   
(130bf0cafb548a89, supertr ll will live forever...       0.32527   

                                                    component_35  \
(04bbc7a4ebe7919a, about einstein my search of ...      -0.15485   
(0c7c341727488579, do go fuck off bastard do yy...      -0.68770   
(130bf0cafb548a89, supertr ll will live forever...      -0.19369   

                                                    component_36  \
(04bbc7a4ebe7919a, about einstein my search of ...      -0.04162   
(0c7c341727488579, do go fuck off bastard do yy...       1.12707   
(130bf0cafb548a89, supertr ll will live forever...      -0.57568   

                                                    component_37  \
(04bbc7a4ebe7919a, about einstein my search of ...       0.07019   
(0c7c341727488579, do go fuck off bastard do yy...      -0.48718   
(130bf0cafb548a89, supertr ll will live forever...       0.16828   

                                                    component_38  component_39  
(04bbc7a4ebe7919a, about einstein my search of ...      -0.26471      -0.32252  
(0c7c341727488579, do go fuck off bastard do yy...      -0.91849       0.34819  


# Second attempt at a useful model

In [1]:
import os
import gensim

# Path to where the word2vec file lives
google_vec_file = '/Users/randy/Documents/GitHub/data_files/GoogleNews-vectors-negative300.bin.gz'

In [2]:
# Load it!  This might take a few minutes...
model = gensim.models.KeyedVectors.load_word2vec_format(google_vec_file, binary=True)

In [3]:
# We can access individual word vectors using a dictionary-like syntax
model['Metis']

array([-3.32031250e-02, -4.98046875e-01,  6.12792969e-02,  3.30078125e-01,
        2.19726562e-01, -2.46093750e-01,  1.03515625e-01,  5.78613281e-02,
       -2.48046875e-01, -2.03125000e-01,  5.51757812e-02,  1.64062500e-01,
        4.34570312e-02, -5.54687500e-01,  1.45507812e-01, -1.32812500e-01,
        2.26562500e-01,  3.71093750e-02, -4.66308594e-02, -1.43554688e-01,
       -1.39648438e-01,  7.27539062e-02, -2.04101562e-01, -1.04492188e-01,
        5.24902344e-02,  1.17187500e-01, -4.76562500e-01, -1.44653320e-02,
        2.69531250e-01, -2.55859375e-01, -5.49316406e-02,  2.41210938e-01,
       -1.62109375e-01,  1.51367188e-01,  9.71679688e-02, -2.92968750e-01,
       -6.44531250e-02,  2.67578125e-01,  7.76367188e-02, -5.42968750e-01,
        1.04003906e-01,  1.97265625e-01,  5.49316406e-02,  6.22558594e-02,
        1.04003906e-01, -3.49609375e-01, -1.95312500e-01,  1.40625000e-01,
       -5.79833984e-03, -1.26953125e-01,  4.17480469e-02, -3.43750000e-01,
       -3.26171875e-01,  

In [4]:
# Some cool results!
model.most_similar('gal' ,topn=8)

[('babe', 0.6436853408813477),
 ('gals', 0.6363939046859741),
 ('cutie', 0.6271624565124512),
 ('hottie', 0.6227655410766602),
 ('hippie_chick', 0.604447066783905),
 ('kinda_gal', 0.6001110672950745),
 ('minx', 0.5954605340957642),
 ('blonde', 0.5936496257781982)]

In [8]:
model.most_similar('dude' ,topn=8)

[('guy', 0.7975313663482666),
 ('dudes', 0.7534066438674927),
 ('motherfucker', 0.7206135988235474),
 ('shit', 0.6915427446365356),
 ('fella', 0.6824806332588196),
 ('bro', 0.6752575039863586),
 ('ass', 0.6721323728561401),
 ('Dude', 0.6643366813659668)]

In [5]:
# Here's an analogy task!
# Positive words contribute positively towards the similarity, negative words negatively
model.most_similar(positive=['woman', 'king'], negative=['man']) #man is to king as woman is to...

[('queen', 0.7118192911148071),
 ('monarch', 0.6189674139022827),
 ('princess', 0.5902431607246399),
 ('crown_prince', 0.5499460697174072),
 ('prince', 0.5377321243286133),
 ('kings', 0.5236844420433044),
 ('Queen_Consort', 0.5235945582389832),
 ('queens', 0.5181134343147278),
 ('sultan', 0.5098593235015869),
 ('monarchy', 0.5087411999702454)]

# Third attempt at useful models

In [12]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, cross_val_predict
from scipy.sparse import hstack
from scipy.special import logit, expit

from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF
from sklearn.metrics.pairwise import cosine_similarity

import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import re
import nltk
import string

import gensim

import pickle

from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF
import seaborn as sns
import matplotlib.pyplot as plt
import re
import nltk
import string

%matplotlib inline
#set column widths t
pd.set_option('display.max_colwidth', -1)

# Import and clean data

In [14]:
class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

raw = pd.read_csv('/Users/randy/Documents/GitHub/Twitch_Chat_Harassment/toxic_comment_classification/jigsaw-toxic-comment-classification-challenge/cleaned_train.csv').fillna(' ')
test = pickle.load(open('/Users/randy/Documents/GitHub/Twitch_Chat_Harassment/chat_logs/callofduty_chat.p','rb'))

In [15]:
#check the first few rows. Note that there are a bunch of NaN columns
raw.head(3)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,explanation why the edits made under my username hardcore metallica fan were reverted ? they were not vandalisms just closure on some gas after i voted at new york dolls fac and please do not remove the template from the talk page since i am retired now,0,0,0,0,0,0
1,000103f0d9cfb60f,d aww ! he matches this background colour i am seemingly stuck with thanks (talk) : january (utc),0,0,0,0,0,0
2,000113f07ec002fd,hey man i am really not trying to edit war it just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page he seems to care more about the formatting than the actual info,0,0,0,0,0,0


In [22]:
data = raw.drop(columns=['id'])

In [23]:
data['comment_text'] = data['comment_text'].str.lower()

In [24]:
#check the dimensions
data.shape

(159571, 7)

In [26]:
#remove non-ascii characters
def remove_non_ascii(text):
    text  = "".join([char for char in text if char in string.printable])
    text = re.sub('[0-9]+', '', text)
    return text

data['non_ascii_tweet'] = data['comment_text'].apply(lambda x: remove_non_ascii(x))
data.head(2)

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,non_ascii_tweet
0,explanation why the edits made under my username hardcore metallica fan were reverted ? they were not vandalisms just closure on some gas after i voted at new york dolls fac and please do not remove the template from the talk page since i am retired now,0,0,0,0,0,0,explanation why the edits made under my username hardcore metallica fan were reverted ? they were not vandalisms just closure on some gas after i voted at new york dolls fac and please do not remove the template from the talk page since i am retired now
1,d aww ! he matches this background colour i am seemingly stuck with thanks (talk) : january (utc),0,0,0,0,0,0,d aww ! he matches this background colour i am seemingly stuck with thanks (talk) : january (utc)
2,hey man i am really not trying to edit war it just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page he seems to care more about the formatting than the actual info,0,0,0,0,0,0,hey man i am really not trying to edit war it just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page he seems to care more about the formatting than the actual info
3,more i cannot make any real suggestions on improvement - i wondered if the section statistics should be later on or a subsection of types of accidents - i think the references may need tidying so that they are all in the exact same format ie date format etc i can do that later on if no - one else does first - if you have any preferences for formatting style on references or want to do it yourself please let me know there appears to be a backlog on articles for review so i guess there may be a delay until a reviewer turns up it listed in the relevant form eg wikipedia : good_article_nominations#transport,0,0,0,0,0,0,more i cannot make any real suggestions on improvement - i wondered if the section statistics should be later on or a subsection of types of accidents - i think the references may need tidying so that they are all in the exact same format ie date format etc i can do that later on if no - one else does first - if you have any preferences for formatting style on references or want to do it yourself please let me know there appears to be a backlog on articles for review so i guess there may be a delay until a reviewer turns up it listed in the relevant form eg wikipedia : good_article_nominations#transport
4,you sir are my hero any chance you remember what page that on ?,0,0,0,0,0,0,you sir are my hero any chance you remember what page that on ?


In [27]:
#remove the punctuation
def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    return text

data['no_punct_tweet'] = data['non_ascii_tweet'].apply(lambda x: remove_punct(x))
data.head()

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,non_ascii_tweet,no_punct_tweet
0,explanation why the edits made under my username hardcore metallica fan were reverted ? they were not vandalisms just closure on some gas after i voted at new york dolls fac and please do not remove the template from the talk page since i am retired now,0,0,0,0,0,0,explanation why the edits made under my username hardcore metallica fan were reverted ? they were not vandalisms just closure on some gas after i voted at new york dolls fac and please do not remove the template from the talk page since i am retired now,explanation why the edits made under my username hardcore metallica fan were reverted they were not vandalisms just closure on some gas after i voted at new york dolls fac and please do not remove the template from the talk page since i am retired now
1,d aww ! he matches this background colour i am seemingly stuck with thanks (talk) : january (utc),0,0,0,0,0,0,d aww ! he matches this background colour i am seemingly stuck with thanks (talk) : january (utc),d aww he matches this background colour i am seemingly stuck with thanks talk january utc
2,hey man i am really not trying to edit war it just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page he seems to care more about the formatting than the actual info,0,0,0,0,0,0,hey man i am really not trying to edit war it just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page he seems to care more about the formatting than the actual info,hey man i am really not trying to edit war it just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page he seems to care more about the formatting than the actual info
3,more i cannot make any real suggestions on improvement - i wondered if the section statistics should be later on or a subsection of types of accidents - i think the references may need tidying so that they are all in the exact same format ie date format etc i can do that later on if no - one else does first - if you have any preferences for formatting style on references or want to do it yourself please let me know there appears to be a backlog on articles for review so i guess there may be a delay until a reviewer turns up it listed in the relevant form eg wikipedia : good_article_nominations#transport,0,0,0,0,0,0,more i cannot make any real suggestions on improvement - i wondered if the section statistics should be later on or a subsection of types of accidents - i think the references may need tidying so that they are all in the exact same format ie date format etc i can do that later on if no - one else does first - if you have any preferences for formatting style on references or want to do it yourself please let me know there appears to be a backlog on articles for review so i guess there may be a delay until a reviewer turns up it listed in the relevant form eg wikipedia : good_article_nominations#transport,more i cannot make any real suggestions on improvement i wondered if the section statistics should be later on or a subsection of types of accidents i think the references may need tidying so that they are all in the exact same format ie date format etc i can do that later on if no one else does first if you have any preferences for formatting style on references or want to do it yourself please let me know there appears to be a backlog on articles for review so i guess there may be a delay until a reviewer turns up it listed in the relevant form eg wikipedia goodarticlenominationstransport
4,you sir are my hero any chance you remember what page that on ?,0,0,0,0,0,0,you sir are my hero any chance you remember what page that on ?,you sir are my hero any chance you remember what page that on


In [28]:
#tokenize the data
def tokenization(text):
    text = re.split('\W+', text)
    return text

data['Tweet_tokenized'] = data['no_punct_tweet'].apply(lambda x: tokenization(x.lower()))
data.head()

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,non_ascii_tweet,no_punct_tweet,Tweet_tokenized
0,explanation why the edits made under my username hardcore metallica fan were reverted ? they were not vandalisms just closure on some gas after i voted at new york dolls fac and please do not remove the template from the talk page since i am retired now,0,0,0,0,0,0,explanation why the edits made under my username hardcore metallica fan were reverted ? they were not vandalisms just closure on some gas after i voted at new york dolls fac and please do not remove the template from the talk page since i am retired now,explanation why the edits made under my username hardcore metallica fan were reverted they were not vandalisms just closure on some gas after i voted at new york dolls fac and please do not remove the template from the talk page since i am retired now,"[explanation, why, the, edits, made, under, my, username, hardcore, metallica, fan, were, reverted, they, were, not, vandalisms, just, closure, on, some, gas, after, i, voted, at, new, york, dolls, fac, and, please, do, not, remove, the, template, from, the, talk, page, since, i, am, retired, now]"
1,d aww ! he matches this background colour i am seemingly stuck with thanks (talk) : january (utc),0,0,0,0,0,0,d aww ! he matches this background colour i am seemingly stuck with thanks (talk) : january (utc),d aww he matches this background colour i am seemingly stuck with thanks talk january utc,"[d, aww, he, matches, this, background, colour, i, am, seemingly, stuck, with, thanks, talk, january, utc]"
2,hey man i am really not trying to edit war it just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page he seems to care more about the formatting than the actual info,0,0,0,0,0,0,hey man i am really not trying to edit war it just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page he seems to care more about the formatting than the actual info,hey man i am really not trying to edit war it just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page he seems to care more about the formatting than the actual info,"[hey, man, i, am, really, not, trying, to, edit, war, it, just, that, this, guy, is, constantly, removing, relevant, information, and, talking, to, me, through, edits, instead, of, my, talk, page, he, seems, to, care, more, about, the, formatting, than, the, actual, info]"
3,more i cannot make any real suggestions on improvement - i wondered if the section statistics should be later on or a subsection of types of accidents - i think the references may need tidying so that they are all in the exact same format ie date format etc i can do that later on if no - one else does first - if you have any preferences for formatting style on references or want to do it yourself please let me know there appears to be a backlog on articles for review so i guess there may be a delay until a reviewer turns up it listed in the relevant form eg wikipedia : good_article_nominations#transport,0,0,0,0,0,0,more i cannot make any real suggestions on improvement - i wondered if the section statistics should be later on or a subsection of types of accidents - i think the references may need tidying so that they are all in the exact same format ie date format etc i can do that later on if no - one else does first - if you have any preferences for formatting style on references or want to do it yourself please let me know there appears to be a backlog on articles for review so i guess there may be a delay until a reviewer turns up it listed in the relevant form eg wikipedia : good_article_nominations#transport,more i cannot make any real suggestions on improvement i wondered if the section statistics should be later on or a subsection of types of accidents i think the references may need tidying

In [29]:
#remove STOPWORDS
stopword = nltk.corpus.stopwords.words('english')
#stopword.extend(['yr', 'year', 'woman', 'man', 'girl','boy','one', 'two', 'sixteen', 'yearold', 'fu', 'weeks', 'week',
#               'treatment', 'associated', 'patients', 'may','day', 'case','old'])


In [30]:
def remove_stopwords(text):
    text = [word for word in text if word not in stopword]
    return text
    
data['Tweet_nonstop'] = data['Tweet_tokenized'].apply(lambda x: remove_stopwords(x))
data.head(5)

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,non_ascii_tweet,no_punct_tweet,Tweet_tokenized,Tweet_nonstop
0,explanation why the edits made under my username hardcore metallica fan were reverted ? they were not vandalisms just closure on some gas after i voted at new york dolls fac and please do not remove the template from the talk page since i am retired now,0,0,0,0,0,0,explanation why the edits made under my username hardcore metallica fan were reverted ? they were not vandalisms just closure on some gas after i voted at new york dolls fac and please do not remove the template from the talk page since i am retired now,explanation why the edits made under my username hardcore metallica fan were reverted they were not vandalisms just closure on some gas after i voted at new york dolls fac and please do not remove the template from the talk page since i am retired now,"[explanation, why, the, edits, made, under, my, username, hardcore, metallica, fan, were, reverted, they, were, not, vandalisms, just, closure, on, some, gas, after, i, voted, at, new, york, dolls, fac, and, please, do, not, remove, the, template, from, the, talk, page, since, i, am, retired, now]","[explanation, edits, made, username, hardcore, metallica, fan, reverted, vandalisms, closure, gas, voted, new, york, dolls, fac, please, remove, template, talk, page, since, retired]"
1,d aww ! he matches this background colour i am seemingly stuck with thanks (talk) : january (utc),0,0,0,0,0,0,d aww ! he matches this background colour i am seemingly stuck with thanks (talk) : january (utc),d aww he matches this background colour i am seemingly stuck with thanks talk january utc,"[d, aww, he, matches, this, background, colour, i, am, seemingly, stuck, with, thanks, talk, january, utc]","[aww, matches, background, colour, seemingly, stuck, thanks, talk, january, utc]"
2,hey man i am really not trying to edit war it just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page he seems to care more about the formatting than the actual info,0,0,0,0,0,0,hey man i am really not trying to edit war it just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page he seems to care more about the formatting than the actual info,hey man i am really not trying to edit war it just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page he seems to care more about the formatting than the actual info,"[hey, man, i, am, really, not, trying, to, edit, war, it, just, that, this, guy, is, constantly, removing, relevant, information, and, talking, to, me, through, edits, instead, of, my, talk, page, he, seems, to, care, more, about, the, formatting, than, the, actual, info]","[hey, man, really, trying, edit, war, guy, constantly, removing, relevant, information, talking, edits, instead, talk, page, seems, care, formatting, actual, info]"
3,more i cannot make any real suggestions on improvement - i wondered if the section statistics should be later on or a subsection of types of accidents - i think the references may need tidying so that they are all in the exact same format ie date format etc i can do that later on if no - one else does first - if you have any preferences for formatting style on references or want to do it yourself please let me know there appears to be a backlog on articles for review so i guess there may be a delay until a reviewer turns up it listed in the relevant form eg wikipedia : good_article_nominations#transport,0,0,0,0,0,0,more i cannot make any real suggestions on improvement - i wondered if the section statistics should be later on or a subsection of types of accidents - i think the references may need tidying so that they are all in the exact same format ie date format etc i can do that later on if no - one else does first - if you have any preferences for formatti

In [31]:
#Stemming and Lammitization
ps = nltk.PorterStemmer()

def stemming(text):
    text = [ps.stem(word) for word in text if len(text) <30]
    return text

data['Tweet_stemmed'] = data['Tweet_nonstop'].apply(lambda x: stemming(x))
data.head()

RecursionError: maximum recursion depth exceeded in comparison

In [33]:
wn = nltk.WordNetLemmatizer()

def lemmatizer(text):
    text = [wn.lemmatize(word) for word in text if len(word) <30]
    return text

data['Tweet_lemmatized'] = data['Tweet_nonstop'].apply(lambda x: lemmatizer(x))


,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,non_ascii_tweet,no_punct_tweet,Tweet_tokenized,Tweet_nonstop,Tweet_lemmatized
0,explanation why the edits made under my username hardcore metallica fan were reverted ? they were not vandalisms just closure on some gas after i voted at new york dolls fac and please do not remove the template from the talk page since i am retired now,0,0,0,0,0,0,explanation why the edits made under my username hardcore metallica fan were reverted ? they were not vandalisms just closure on some gas after i voted at new york dolls fac and please do not remove the template from the talk page since i am retired now,explanation why the edits made under my username hardcore metallica fan were reverted they were not vandalisms just closure on some gas after i voted at new york dolls fac and please do not remove the template from the talk page since i am retired now,"[explanation, why, the, edits, made, under, my, username, hardcore, metallica, fan, were, reverted, they, were, not, vandalisms, just, closure, on, some, gas, after, i, voted, at, new, york, dolls, fac, and, please, do, not, remove, the, template, from, the, talk, page, since, i, am, retired, now]","[explanation, edits, made, username, hardcore, metallica, fan, reverted, vandalisms, closure, gas, voted, new, york, dolls, fac, please, remove, template, talk, page, since, retired]","[explanation, edits, made, username, hardcore, metallica, fan, reverted, vandalism, closure, gas, voted, new, york, doll, fac, please, remove, template, talk, page, since, retired]"
1,d aww ! he matches this background colour i am seemingly stuck with thanks (talk) : january (utc),0,0,0,0,0,0,d aww ! he matches this background colour i am seemingly stuck with thanks (talk) : january (utc),d aww he matches this background colour i am seemingly stuck with thanks talk january utc,"[d, aww, he, matches, this, background, colour, i, am, seemingly, stuck, with, thanks, talk, january, utc]","[aww, matches, background, colour, seemingly, stuck, thanks, talk, january, utc]","[aww, match, background, colour, seemingly, stuck, thanks, talk, january, utc]"
2,hey man i am really not trying to edit war it just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page he seems to care more about the formatting than the actual info,0,0,0,0,0,0,hey man i am really not trying to edit war it just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page he seems to care more about the formatting than the actual info,hey man i am really not trying to edit war it just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page he seems to care more about the formatting than the actual info,"[hey, man, i, am, really, not, trying, to, edit, war, it, just, that, this, guy, is, constantly, removing, relevant, information, and, talking, to, me, through, edits, instead, of, my, talk, page, he, seems, to, care, more, about, the, formatting, than, the, actual, info]","[hey, man, really, trying, edit, war, guy, constantly, removing, relevant, information, talking, edits, instead, talk, page, seems, care, formatting, actual, info]","[hey, man, really, trying, edit, war, guy, constantly, removing, relevant, information, talking, edits, instead, talk, page, seems, care, formatting, actual, info]"
3,more i cannot make any real suggestions on improvement - i wondered if the section statistics should be later on or a subsection of types of accidents - i think the references may need tidying so that they are all in the exact same format ie date format etc i can do that later on if no - one else does first - if you have any preferences for formatting style on references or want to do it yourself please let me know there appears to be a backlog on articles for review so i guess there may be a delay until a reviewer turns 

In [34]:
data.head(20)

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,non_ascii_tweet,no_punct_tweet,Tweet_tokenized,Tweet_nonstop,Tweet_lemmatized
0,explanation why the edits made under my username hardcore metallica fan were reverted ? they were not vandalisms just closure on some gas after i voted at new york dolls fac and please do not remove the template from the talk page since i am retired now,0,0,0,0,0,0,explanation why the edits made under my username hardcore metallica fan were reverted ? they were not vandalisms just closure on some gas after i voted at new york dolls fac and please do not remove the template from the talk page since i am retired now,explanation why the edits made under my username hardcore metallica fan were reverted they were not vandalisms just closure on some gas after i voted at new york dolls fac and please do not remove the template from the talk page since i am retired now,"[explanation, why, the, edits, made, under, my, username, hardcore, metallica, fan, were, reverted, they, were, not, vandalisms, just, closure, on, some, gas, after, i, voted, at, new, york, dolls, fac, and, please, do, not, remove, the, template, from, the, talk, page, since, i, am, retired, now]","[explanation, edits, made, username, hardcore, metallica, fan, reverted, vandalisms, closure, gas, voted, new, york, dolls, fac, please, remove, template, talk, page, since, retired]","[explanation, edits, made, username, hardcore, metallica, fan, reverted, vandalism, closure, gas, voted, new, york, doll, fac, please, remove, template, talk, page, since, retired]"
1,d aww ! he matches this background colour i am seemingly stuck with thanks (talk) : january (utc),0,0,0,0,0,0,d aww ! he matches this background colour i am seemingly stuck with thanks (talk) : january (utc),d aww he matches this background colour i am seemingly stuck with thanks talk january utc,"[d, aww, he, matches, this, background, colour, i, am, seemingly, stuck, with, thanks, talk, january, utc]","[aww, matches, background, colour, seemingly, stuck, thanks, talk, january, utc]","[aww, match, background, colour, seemingly, stuck, thanks, talk, january, utc]"
2,hey man i am really not trying to edit war it just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page he seems to care more about the formatting than the actual info,0,0,0,0,0,0,hey man i am really not trying to edit war it just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page he seems to care more about the formatting than the actual info,hey man i am really not trying to edit war it just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page he seems to care more about the formatting than the actual info,"[hey, man, i, am, really, not, trying, to, edit, war, it, just, that, this, guy, is, constantly, removing, relevant, information, and, talking, to, me, through, edits, instead, of, my, talk, page, he, seems, to, care, more, about, the, formatting, than, the, actual, info]","[hey, man, really, trying, edit, war, guy, constantly, removing, relevant, information, talking, edits, instead, talk, page, seems, care, formatting, actual, info]","[hey, man, really, trying, edit, war, guy, constantly, removing, relevant, information, talking, edits, instead, talk, page, seems, care, formatting, actual, info]"
3,more i cannot make any real suggestions on improvement - i wondered if the section statistics should be later on or a subsection of types of accidents - i think the references may need tidying so that they are all in the exact same format ie date format etc i can do that later on if no - one else does first - if you have any preferences for formatting style on references or want to do it yourself please let me know there appears to be a backlog on articles for review so i guess there may be a delay until a reviewer turns 

In [35]:
def clean_text(text):
    text_lc = "".join([word.lower() for word in text if word not in string.punctuation]) # remove puntuation
    text_rc = re.sub('[0-9]+', '', text_lc)
    tokens = re.split('\W+', text_rc)    # tokenization
    text = [ps.stem(word) for word in tokens if word not in stopword]  # remove stopwords and stemming
    return text

# Step 2: Vectorize (create countVectorizer and TF-IDF Vectorizer)

# Starting with countVectorizer:

In [40]:
countVectorizer = CountVectorizer(analyzer=clean_text, ngram_range=(3,3), stop_words='english') 
X = countVectorizer.fit_transform(data['comment_text'])
# print('{} Number of tweets has {} words'.format(X.shape[0], X.shape[1]))
print(f"Dimensions of Document-term matrix: {X.toarray().shape}")
# print(X.get_feature_names())

RecursionError: maximum recursion depth exceeded in comparison

In [ ]:
count_vect_df = pd.DataFrame(X.toarray(), columns=countVectorizer.get_feature_names())
count_vect_df.head()

# TF-IDF Vectorizer (sklearn method):

In [41]:
# For sklearn, it's VERY similar to how we did CountVectorizer
vectorizer = TfidfVectorizer(analyzer=clean_text, ngram_range=(1,1), stop_words='english')
X_tfidf = vectorizer.fit_transform(data['comment_text'])

RecursionError: maximum recursion depth exceeded in comparison

In [42]:
# Tokenize the documents
# Convert a document into a list of lowercase tokens.
tokenized_docs = [gensim.utils.simple_preprocess(d) for d in data['comment_text']]

# Create a Gensim Dictionary.  This creates an id to word mapping for everything in our vocbulary
# It is NOT the same as the dictionary object in the Python standard library
# index is key, value is word
mydict = gensim.corpora.Dictionary()

# Create a Gensim Corpus object.  This creates a list of tuples for each document.
# The first element of the tuple is the word id, the second is the number of counts
mycorpus = [mydict.doc2bow(doc, allow_update=True) for doc in tokenized_docs]

In [ ]:
# This creates the doc-term matrix as a numpy array.
# Typically these matrices are HUGE so, it's usuall not a great idea to create the full dense doc-term matrix.
# We do it here to illustrate that you can get the same info as we obtained in scikit-learn!
doc_term_matrix = gensim.matutils.corpus2dense(mycorpus, num_terms=len(mydict))

In [ ]:
doc_term_matrix

In [ ]:
# Creating a tf-idf model is very simple!
tfidf = gensim.models.TfidfModel(mycorpus)
tfidf_matrix = gensim.matutils.corpus2dense(tfidf[mycorpus], num_terms=len(mydict))

In [ ]:
tfidf_matrix

# Train Models

# ExtraTreeClassifier

In [12]:
from sklearn.ensemble import ExtraTreesClassifier

losses = []
predictions = {'id': test['id']}
models = {}
for class_name in class_names:
    train_target = train[class_name]
    classifier = ExtraTreesClassifier(n_estimators=30)

    cv_loss = np.mean(cross_val_score(classifier, train_features, train_target, cv=3, scoring='roc_auc'))
    losses.append(cv_loss)
    print('CV score for class {} is {}'.format(class_name, cv_loss))
    
    classifier.fit(train_features, train_target)
    models[class_name] = classifier
    predictions[class_name] = classifier.predict_proba(test_features)[:, 1]

CV score for class toxic is 0.9533489165216545
CV score for class severe_toxic is 0.9407712280477519
CV score for class obscene is 0.9780868191286545
CV score for class threat is 0.8594060404950042
CV score for class insult is 0.9618170899384982
CV score for class identity_hate is 0.9042010302886488


In [69]:
test_features.toarray()

array([[0.08864541, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.06752337, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.06550807, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.06367916, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.08214958, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.07227823, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [63]:
check = [['message','id','toxic','severe_toxic','obscene','threat','insult','identity_hate']]

for item in zip(*predictions.values()):
    check.append([list_sentences_test[item[0]],item[0],item[1],item[2],item[3],item[4],item[5],item[6]])

In [64]:
x = pd.DataFrame(check)

In [65]:
new_header = x.iloc[0] #grab the first row for the header
x = x[1:] #take the data less the header row
x.columns = new_header #set the header row as the df header

In [70]:
x.shape

(1208, 8)

In [48]:
x.columns

Int64Index([0, 1, 2, 3, 4, 5, 6, 7], dtype='int64')

In [71]:
models

{'toxic': ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=30, n_jobs=None,
            oob_score=False, random_state=None, verbose=0, warm_start=False),
 'severe_toxic': ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=30, n_jobs=None,
            oob_score=False, random_state=None, verbose=0, warm_start=False),
 'obscene': ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_node

In [17]:
predictions.keys()

dict_keys(['id', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'])

In [72]:
#pickle the models
# Save Model as a pickle Using joblib
import pickle
from sklearn.externals import joblib

# Save the model as a pickle in a file 
joblib.dump(models, 'twitch_models.p') 

# Load the model from the file 
pickled_models = joblib.load('twitch_models.p')  
  


In [ ]:
pickled_models['toxic'].fit(train_features, train_target)
predictions['toxic'] = pickled_models['toxic'].predict_proba(test_features)[:, 1]

In [ ]:
toxic = pickled_models['toxic']

In [ ]:
cv_loss = np.mean(cross_val_score(toxic, train_features, train_target, cv=3, scoring='roc_auc'))
print('CV score for toxic class is {}'.format(cv_loss))